In [ ]:
# Travel Insurance데이터 세트
# 고객별 여행보험 가입 여부 예측 모델
# 가장 높은 Accuracy값을 가지는 최종 모델을 제출
# 결과값은 csv파일로 제출
# 모델의 성능은 ROC-AUC 평가지표

In [1]:
import pandas as pd

train = pd.read_csv('2_1_travel_insurance_train.csv')
test = pd.read_csv('2_2_travel_insurance_test.csv')

In [2]:
train.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,1704,26,Private Sector/Self Employed,Yes,1400000,3,1,No,Yes,0
1,491,28,Private Sector/Self Employed,Yes,1100000,4,1,No,No,1
2,414,33,Private Sector/Self Employed,Yes,1400000,4,0,No,Yes,1
3,120,28,Private Sector/Self Employed,Yes,800000,3,1,No,No,0
4,1268,33,Government Sector,Yes,1000000,5,0,No,Yes,1


In [3]:
test.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1569,27,Government Sector,Yes,500000,5,0,No,No
1,1344,25,Private Sector/Self Employed,Yes,1700000,3,0,Yes,No
2,1429,32,Government Sector,Yes,650000,3,0,No,No
3,896,33,Government Sector,Yes,600000,4,0,No,No
4,101,33,Private Sector/Self Employed,Yes,1500000,3,1,Yes,Yes


In [5]:
X_train = train.drop(columns='TravelInsurance')
y_train = train[['ID', 'TravelInsurance']]
X_test = test.copy()
print(X_train.shape, y_train.shape, X_test.shape)

(1490, 9) (1490, 2) (497, 9)


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB


In [7]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


In [8]:
int_train = X_train.select_dtypes(exclude='object')
object_train = X_train.select_dtypes(include='object')
int_test = X_test.select_dtypes(exclude='object')
object_test = X_test.select_dtypes(include='object')
int_train.head()

,ID,Age,AnnualIncome,FamilyMembers,ChronicDiseases
0,1704,26,1400000,3,1
1,491,28,1100000,4,1
2,414,33,1400000,4,0
3,120,28,800000,3,1
4,1268,33,1000000,5,0


In [9]:
object_train.head()

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
0,Private Sector/Self Employed,Yes,No,Yes
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,Yes
3,Private Sector/Self Employed,Yes,No,No
4,Government Sector,Yes,No,Yes


In [10]:
int_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ID               1490 non-null   int64
 1   Age              1490 non-null   int64
 2   AnnualIncome     1490 non-null   int64
 3   FamilyMembers    1490 non-null   int64
 4   ChronicDiseases  1490 non-null   int64
dtypes: int64(5)
memory usage: 58.3 KB


In [11]:
object_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Employment Type      497 non-null    object
 1   GraduateOrNot        497 non-null    object
 2   FrequentFlyer        497 non-null    object
 3   EverTravelledAbroad  497 non-null    object
dtypes: object(4)
memory usage: 15.7+ KB


In [13]:
# MinMax scale
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# int_train의 4개 컬럼에 대해
cols = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']

display(int_train.head())
# int_train 데이터프레임의 해당 컬럼들을 MinMaxScaler로 스케일링
int_train[cols] = scaler.fit_transform(int_train[cols])
# int_test 데이터프레임의 해당 컬럼들도 MinMaxScaler로 스케일링
int_test[cols] = scaler.fit_transform(int_test[cols])
# int_train의 첫 5행을 출력
int_train.head()

,ID,Age,AnnualIncome,FamilyMembers,ChronicDiseases
0,1704,0.1,0.733333,0.142857,1.0
1,491,0.3,0.533333,0.285714,1.0
2,414,0.8,0.733333,0.285714,0.0
3,120,0.3,0.333333,0.142857,1.0
4,1268,0.8,0.466667,0.428571,0.0


,ID,Age,AnnualIncome,FamilyMembers,ChronicDiseases
0,1704,0.1,0.733333,0.142857,1.0
1,491,0.3,0.533333,0.285714,1.0
2,414,0.8,0.733333,0.285714,0.0
3,120,0.3,0.333333,0.142857,1.0
4,1268,0.8,0.466667,0.428571,0.0


In [14]:
display(object_train.head())
object_train = pd.get_dummies(object_train)
object_test = pd.get_dummies(object_test)
object_train.head()

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
0,Private Sector/Self Employed,Yes,No,Yes
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,Yes
3,Private Sector/Self Employed,Yes,No,No
4,Government Sector,Yes,No,Yes


,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,False,True,False,True,True,False,False,True
1,False,True,False,True,True,False,True,False
2,False,True,False,True,True,False,False,True
3,False,True,False,True,True,False,True,False
4,True,False,False,True,True,False,False,True


In [15]:
# 데이터 결합하기
X_train = pd.concat([int_train, object_train], axis=1)
X_test = pd.concat([int_test, object_test], axis=1)
print(X_train.shape, X_test.shape)
X_train.head()

(1490, 13) (497, 13)


,ID,Age,AnnualIncome,FamilyMembers,ChronicDiseases,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,1704,0.1,0.733333,0.142857,1.0,False,True,False,True,True,False,False,True
1,491,0.3,0.533333,0.285714,1.0,False,True,False,True,True,False,True,False
2,414,0.8,0.733333,0.285714,0.0,False,True,False,True,True,False,False,True
3,120,0.3,0.333333,0.142857,1.0,False,True,False,True,True,False,True,False
4,1268,0.8,0.466667,0.428571,0.0,True,False,False,True,True,False,False,True


In [16]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['TravelInsurance'],
                                            test_size=0.1,
                                            random_state=1234)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1341, 13), (149, 13), (1341,), (149,))

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [19]:
# 로지스틱 회귀
model_lgm = LogisticRegression()
model_lgm.fit(X_tr, y_tr)
pred_lgm=model_lgm.predict_proba(X_val)
roc_auc_score(y_val, pred_lgm[:,1])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7450676982591876

In [20]:
# 서포트벡터머신
model_svm = SVC(probability=True)
model_svm.fit(X_tr, y_tr)
pred_svm = model_svm.predict_proba(X_val)
roc_auc_score(y_val, pred_svm[:,1])

0.7075435203094778

In [21]:
# 의사결정나무
model_dt = DecisionTreeClassifier()
model_dt.fit(X_tr, y_tr)
pred_dt = model_dt.predict_proba(X_val)
roc_auc_score(y_val, pred_dt[:,1])

0.7033849129593811

In [23]:
# 랜덤포레스트
model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr)
pred_rf = model_rf.predict_proba(X_val)
roc_auc_score(y_val, pred_rf[:,1])

0.7913926499032883

In [26]:
# LightGBM
model_lgb = LGBMClassifier()
model_lgb.fit(X_tr, y_tr)
pred_lgb = model_lgb.predict_proba(X_val)
roc_auc_score(y_val, pred_rf[:,1])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 485, number of negative: 856
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 321
[LightGBM] [Info] Number of data points in the train set: 1341, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.361670 -> initscore=-0.568121
[LightGBM] [Info] Start training from score -0.568121


0.7913926499032883

In [27]:
model_final = RandomForestClassifier()
model_final.fit(X_train, y_train['TravelInsurance'])
pred_final = model_final.predict_proba(X_test)
pred_final[:5]


array([[0.84, 0.16],
       [0.01, 0.99],
       [0.79, 0.21],
       [0.86, 0.14],
       [0.11, 0.89]])

In [28]:
submission = pd.DataFrame({
    "index" : X_test.index,
    "y_pred": pred_final[:,1]
})
submission.head()

,index,y_pred
0,0,0.16
1,1,0.99
2,2,0.21
3,3,0.14
4,4,0.89


In [29]:
submission.to_csv("submission.csv", index=False)